In [16]:
import requests
import pandas as pd
import numpy as np
import xarray as xr
import sqlite3
import os
from datetime import datetime
import cdsapi
import csv

In [17]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('../data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table : ", error)

# Fonction pour lancer les requêtes

In [18]:
def get_on_cds_api(dataframe_piezo, piezo, path):
    # Initialisation de la connexion à l'API CDS
    # wait_until_complete = False : le programme Python n'attend pas que l'API réponde (issu de la libraie asyncio)
    # delete = False : le fichier créé sur le serveur source n'est pas supprimé automatiquement 
    
    date_debut = dataframe_piezo.loc[piezo,'date_debut'].year
    date_fin = dataframe_piezo.loc[piezo,'date_fin'].year
    
    # La limite du nombre de données est de 120000. Avec 10 variables, 31 jours, et 12 mois, on n epeut récupérer que 32 années.
    # Si l'interval d'année est supérieur à 32, on réduit la plage de temps à date de fin - 31 ans pour avoir 32 années max.
    if date_fin - date_debut >= 32:
        date_debut = date_fin - 31
    
    c = cdsapi.Client(wait_until_complete=False, delete=False)
    
    # Données à récupérer auprès de l'API
    r = c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': [
                    '2m_temperature', 'convective_precipitation', 'convective_rain_rate',
                    'convective_snowfall', 'large_scale_rain_rate', 'large_scale_snowfall', 'snowfall',
                    'large_scale_snowfall_rate_water_equivalent', 'potential_evaporation',
                    'total_precipitation',
                ],
                'year': [str(date) for date in (np.arange(date_debut, date_fin))],
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': '00:00',
                'area': [
                    dataframe_piezo.loc[piezo,"latitude"], dataframe_piezo.loc[piezo,"longitude"], dataframe_piezo.loc[piezo,"latitude"], dataframe_piezo.loc[piezo,"longitude"],
                ],
            })    
    return r

# Connexion à la bdd piezo
Construction d'un dataframe 'df_piezo' à partir de la requête sur la bdd

In [19]:
region = "BRE"

In [20]:
connexion = sqlite3.connect('../data/liste_piezos.db')
curseur = connexion.cursor()
requete = f"""
    SELECT code_bss, date_debut, date_fin, nb_mesures_piezo, latitude, longitude 
        FROM data_piezo
    INNER JOIN code_region
    ON code_dpt = departement
    WHERE code_region IS '{region}'
    AND date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""

df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])
df_piezo.head()

,date_debut,date_fin,nb_mesures_piezo,latitude,longitude
code_bss,,,,,
02388X0060_F1,1993-07-29,2022-05-15,9747,48.459156,-4.350352
03851X0021_S2,1989-01-10,2022-05-15,10130,47.790161,-2.817043
03454X0070_F,1993-08-03,2022-05-15,8593,48.000530,-4.335488
03128X0011_F,1980-02-01,2022-05-16,13601,48.089266,-3.276193
02442X0111_F,2004-11-18,2022-05-15,5959,48.521916,-2.418250


## Lancer les requêtes auprès de l'API
!!! NE LANCER QU'UNE SEULE FOIS POUR NE PAS CREER DES DOUBLONS DE REQUETES !!!

Un dictionnaire est créé avec en clé le code_bss du piezo et en valeur l'id de la requête

Ne pas lancer la cellule ci-dessous si des requetes sont en cours. 

In [21]:
# Parcourir les codes_bss des piezos dans la liste précédemment créé :
dict_bss = {}

go_requete = input("""
            Voulez-vous lancer une série de nouvelles requêtes ? /n 
            Des requêtes pourraient se retrouver en double. /n 
            Pour lancer les requêtes entrez 'req' et entrée. /n
            Sinon, appuyez simplement sur entrée.
""")
                   
if go_requete == 'req' :
                      
    for piezo in df_piezo.index:
        path_piezo = f'../data/era5/{piezo}'
        path_nc = f"{path_piezo}.nc"
        path_pkl = f"{path_piezo}.pkl"

        if os.path.isfile(path_nc) : #si le fichier nc existe on le converti en pkl
            print(f"Le fichier {path_nc} existe.")
            df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
            print(f"Fichier {piezo}.nc converti en fichier pickle")
            os.remove(path_nc)
            print(f"Fichier {path_nc} supprimé")

        elif os.path.isfile(path_pkl) : #si le fichier pkl existe, on affiche qu'il existe
            print(f"Le fichier {piezo}.pkl existe déjà.")
            # puis on le supprime de la liste
            df_piezo.drop(index=piezo, axis=1)

        else : # Sinon on lance une requête auprès de l'API
            result = get_on_cds_api(df_piezo, piezo, path_nc)
            # récupération de l'id de la requête
            dict_bss[piezo] = result.reply["request_id"]
            print(f"{datetime.now()} : chargement du fichier {piezo}.nc en cours ({dict_bss[piezo]}).")
            
            # Enregistrement de l'id de requete dans un fichier csv
            if os.path.isfile('../data/hist_requetes.csv'): # Si le fichier existe                
                with open('../data/hist_requetes.csv', 'a', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
            else : # Si le fichier n'existe pas
                with open('../data/hist_requetes.csv', 'w', encoding='utf-8', newline='') as f:  
                    writer = csv.writer(f, delimiter = ";")
                    writer.writerow(["code_bss", "id_requete"])
                    for k, v in dict_bss.items():
                        writer.writerow((k, v))
else : 
    print("Aucune requête n'a été lancée.")

2022-07-29 13:15:20,150 INFO Welcome to the CDS
2022-07-29 13:15:20,151 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,242 INFO Welcome to the CDS
2022-07-29 13:15:20,243 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,330 INFO Welcome to the CDS
2022-07-29 13:15:20,331 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-29 13:15:20.200332 : chargement du fichier 02388X0060_F1.nc en cours (036c89bf-b5c7-43ed-8aa6-a87ab5ad0f66).
2022-07-29 13:15:20.290059 : chargement du fichier 03851X0021_S2.nc en cours (a03fa8a5-3606-4c5b-952d-e4695d45746f).
2022-07-29 13:15:20.391684 : chargement du fichier 03454X0070_F.nc en cours (d632345f-2c57-46d8-b9f1-66d947dbd963).


2022-07-29 13:15:20,435 INFO Welcome to the CDS
2022-07-29 13:15:20,436 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,552 INFO Welcome to the CDS
2022-07-29 13:15:20,553 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,637 INFO Welcome to the CDS
2022-07-29 13:15:20,638 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,722 INFO Welcome to the CDS


2022-07-29 13:15:20.513054 : chargement du fichier 03128X0011_F.nc en cours (bffa3546-b5de-4f96-a664-77b25f86f2b3).
2022-07-29 13:15:20.598098 : chargement du fichier 02442X0111_F.nc en cours (df7ad847-b081-4404-8866-f5e5d177715a).
2022-07-29 13:15:20.683432 : chargement du fichier 03822X0021_F.nc en cours (40512165-d955-404a-804d-99b5bd7493fd).


2022-07-29 13:15:20,723 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,822 INFO Welcome to the CDS
2022-07-29 13:15:20,823 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:20,925 INFO Welcome to the CDS
2022-07-29 13:15:20,926 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


...

2022-07-29 13:15:24.842688 : chargement du fichier 03103X0047_PZ.nc en cours (1d0f63ca-c245-4235-ac54-bd93b83e7826).
2022-07-29 13:15:24.927460 : chargement du fichier 03862X0057_PZ.nc en cours (0c29318c-15d3-4cdb-ac38-37e9a4a787fa).


2022-07-29 13:15:25,037 INFO Welcome to the CDS
2022-07-29 13:15:25,038 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:25,096 INFO Welcome to the CDS
2022-07-29 13:15:25,097 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:25,168 INFO Welcome to the CDS
2022-07-29 13:15:25,169 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:25,237 INFO Welcome to the CDS
2022-07-29 13:15:25,238 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-29 13:15:25.070225 : chargement du fichier 02812X0238_PZ.nc en cours (a7ee34b2-5633-4518-b0e7-6dc8d1afa59c).
2022-07-29 13:15:25.140777 : chargement du fichier 03175X0338_PZ.nc en cours (fa7aa9a2-b52d-42c3-9447-c9ec95ebf6ef).
2022-07-29 13:15:25.209593 : chargement du fichier 03172X0088_PZ.nc en cours (0aa9d7ca-b89d-48c3-a3cc-3af7a16cab66).
2022-07-29 13:15:25.270408 : chargement du fichier 03473X0029_PZ.nc en cours (f6a5890d-d48f-489e-ad9c-7cdfbc0d5fb5).


2022-07-29 13:15:25,296 INFO Welcome to the CDS
2022-07-29 13:15:25,296 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-07-29 13:15:25,355 INFO Welcome to the CDS
2022-07-29 13:15:25,356 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


2022-07-29 13:15:25.329251 : chargement du fichier 03162X0049_PZ.nc en cours (c4aa347a-47cb-4a9b-8871-0cab88890291).
2022-07-29 13:15:25.389092 : chargement du fichier 03834X0049_PZ.nc en cours (53168ea3-3619-4118-aad8-57650813c2df).


## Récupérer automatiquement les requêtes abouties 
Peut être lancé à tout moment pour récupérer des fichiers téléchargés 

In [7]:
# Chargement du dictionnaire à partir du csv d'historiques de requetes
with open('../data/hist_requetes.csv', mode='r') as f:
    reader = csv.reader(f, delimiter=";")
    # Permet de sortir la ligne d'en-tête du dictionnaire
    headings = next(reader)
    dict_bss_from_csv = {rows[0]:rows[1] for rows in reader}

nbre_fichier_OK = 0

    
for piezo in dict_bss_from_csv.copy().keys():
    path_piezo = '../data/era5/' + piezo
    path_nc = f"./{path_piezo}.nc"
    path_pkl = f"{path_piezo}.pkl"

    if not os.path.isfile(path_nc) :
        try:
            reply = dict(request_id=dict_bss_from_csv[piezo]) # request_id from above
            new_client = cdsapi.Client(wait_until_complete=False, delete=False)
            result = cdsapi.api.Result(new_client, reply)
            result.update()
            reply = result.reply
            if reply['state'] == 'completed':
                nbre_fichier_OK += 1
                result.download(path_nc)
                print(f"{datetime.now()} : Fichier {piezo}.nc téléchargé.")
                df = xr.open_dataset(path_nc).to_dataframe().reset_index().to_pickle(path=path_pkl)
                print(f"Fichier {piezo}.nc converti en fichier pickle")
                os.remove(path_nc)
                print(f"Fichier {path_nc}.nc supprimé")
                # Si le fichier a été téléchargé on le supprime de la liste du dictionnaire
                del dict_bss[piezo]
        except:
            #print(f"Récupération de l'ID {dict_bss_from_csv[piezo]} impossible.")
            pass
if nbre_fichier_OK == 0:
    print("Aucun fichier de récupéré.")
else :
    print(f"{nbre_fichier_OK} fichiers ont été téléchargés.")

# Mise à jour de la base de donnéees (champ fichier_ERA5)
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"../data/era5") if _.endswith(r".pkl")], name="era5")
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)
print("Base de données à jour")

2022-07-29 13:07:25,995 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data0/adaptor.mars.internal-1658993601.577793-28585-11-f497fb76-830b-4230-8924-7a2db35b8c9a.nc to ./../data/era5/02516X0071_P3-90.nc (200.3K)
2022-07-29 13:07:26,199 INFO Downloading https://download-0001-clone.copernicus-climate.eu/cache-compute-0001/cache/data8/adaptor.mars.internal-1658993831.2944443-22766-4-ec86c08f-f5fc-4e6f-b859-be066db0f422.nc to ./../data/era5/01446X0005_S3-91.nc (174.6K)
2022-07-29 13:07:26,446 INFO Download rate 710K/s
2022-07-29 13:07:26,498 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data0/adaptor.mars.internal-1658993831.2872121-15104-19-38fbb010-068a-48a5-9e24-baa0cfbb9bc4.nc to ./../data/era5/01506X0016_P1.nc (183.2K)


2022-07-29 13:07:26.446779 : Fichier 01446X0005_S3-91.nc téléchargé.
Fichier 01446X0005_S3-91.nc converti en fichier pickle
Fichier ./../data/era5/01446X0005_S3-91.nc.nc supprimé


2022-07-29 13:07:26,761 INFO Download rate 698.7K/s
2022-07-29 13:07:26,809 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data9/adaptor.mars.internal-1658993302.577787-12118-18-811f0cda-9493-4759-bd79-16a8b64b7b24.nc to ./../data/era5/01474X0048_P1.nc (149K)


2022-07-29 13:07:26.762230 : Fichier 01506X0016_P1.nc téléchargé.
Fichier 01506X0016_P1.nc converti en fichier pickle
Fichier ./../data/era5/01506X0016_P1.nc.nc supprimé


2022-07-29 13:07:26,981 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data8/adaptor.mars.internal-1658994565.1918485-22626-10-e4e8efe0-5fc6-4127-ba5c-e12696757db9.nc to ./../data/era5/01801X0010_S1.nc (268.8K)
2022-07-29 13:07:27,171 INFO Downloading https://download-0000-clone.copernicus-climate.eu/cache-compute-0000/cache/data0/adaptor.mars.internal-1658997391.0878623-25423-11-6588043d-9e69-4c1e-b2d8-bd6d566e5997.nc to ./../data/era5/00572X0010_S1.nc (268.8K)
2022-07-29 13:07:27,433 INFO Downloading https://download-0017.copernicus-climate.eu/cache-compute-0017/cache/data4/adaptor.mars.internal-1659003012.3050609-15051-15-59fee1dd-2011-4b2d-a0b1-94fea191ca85.nc to ./../data/era5/01245X0010_S1.nc (268.8K)
2022-07-29 13:07:27,587 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data5/adaptor.mars.internal-1659002240.5829-18625-10-f21fc929-3d07-48d3-9a30-089a02ab80b5.nc to ./../data/era5/02151X0030_

...

2022-07-29 13:08:06.115666 : Fichier 00955X0066_PZ.nc téléchargé.
Fichier 00955X0066_PZ.nc converti en fichier pickle
Fichier ./../data/era5/00955X0066_PZ.nc.nc supprimé


2022-07-29 13:08:06,506 INFO Download rate 458.2K/s
2022-07-29 13:08:06,583 INFO Downloading https://download-0009-clone.copernicus-climate.eu/cache-compute-0009/cache/data3/adaptor.mars.internal-1659087673.5172863-26173-3-dc4b6bae-e625-46bf-b5fa-0fab9fdf7540.nc to ./../data/era5/02531X0029_PZ.nc (183.2K)


2022-07-29 13:08:06.507276 : Fichier 02891X0007_PZ.nc téléchargé.
Fichier 02891X0007_PZ.nc converti en fichier pickle
Fichier ./../data/era5/02891X0007_PZ.nc.nc supprimé


2022-07-29 13:08:06,687 INFO Download rate 1.7M/s


2022-07-29 13:08:06.689588 : Fichier 02531X0029_PZ.nc téléchargé.
Fichier 02531X0029_PZ.nc converti en fichier pickle
Fichier ./../data/era5/02531X0029_PZ.nc.nc supprimé
149 fichiers ont été téléchargés.
Connexion réussie à SQLite
Connexion SQLite est fermée
Base de données à jour


## Jointure era5 et piezo

In [8]:
bss_era5 = pd.Series([_.replace(".pkl","") for _ in os.listdir(r"../data/era5") if _.endswith(r".pkl")], name="era5")
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"../data/piezo") if _.endswith(r".csv")], name="piezo")
piezos_communs = np.intersect1d(bss_era5, bss_piezo)

path = "../data/combined_data/"
# Le fichier doit avoir un minimum de 100 mesures
nb_min_val_com = 100

for piezo in piezos_communs :
    if not os.path.isfile(f'{path}/comb_{piezo}.csv') : #si le fichier n'existe pas 
        df_piezo = pd.read_csv(f"../data/piezo/{piezo}.csv", sep = ";", index_col=0, parse_dates=True)
        # Group_by date_mesure dans le cas de plusieurs mesures le même jour ce qui provoquerait une erreur
        df_piezo = pd.DataFrame(df_piezo.groupby("date_mesure").mean())
        df_piezo = df_piezo.resample('1D').asfreq()
        df_era5 = pd.read_pickle(f"../data/era5/{piezo}.pkl").set_index('time').drop(["latitude", "longitude"], axis=1)
        if "expver" in df_era5.columns :
            df_era5 = df_era5.loc[df_era5["expver"]==1,:].drop("expver", axis=1)
        df = pd.merge(df_piezo, df_era5, how='inner', left_index=True, right_index=True)
        nb_com_obs = df.shape[0]
        if nb_com_obs >= nb_min_val_com :
            df.to_csv(f"../data/combined_data/comb_{piezo}.csv" ,sep=";", index=True)
            print(f"Le fichier comb_{piezo}.csv a été créé et contient {nb_com_obs} obseravtions.")
        else :
            print(f"Le piezo {piezo} ne contient pas assez de données (seulement {nb_com_obs}).")
    else : 
        #print(f"Le fichier comb_{piezo}.csv existe déjà.")
        pass
        
# Mise à jour bdd
requete_fichier_combined = f"""
        UPDATE data_piezo
        SET fichier_combined =
        (CASE WHEN code_bss IN {tuple(piezos_communs)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_combined)
print("Base de données à jour")

Le fichier comb_00435X0079_S1.csv a été créé et contient 11323 obseravtions.
Le fichier comb_00441X0007_P.csv a été créé et contient 7522 obseravtions.
Le fichier comb_00445X0006_P.csv a été créé et contient 7248 obseravtions.
Le fichier comb_00576X0033_P.csv a été créé et contient 5185 obseravtions.
Le fichier comb_00581X0015_P.csv a été créé et contient 7606 obseravtions.
Le fichier comb_00582X0069_P.csv a été créé et contient 7158 obseravtions.
Le fichier comb_00583X0005_S1.csv a été créé et contient 11323 obseravtions.
Le fichier comb_00584X0020_P.csv a été créé et contient 5784 obseravtions.
Le fichier comb_00588X0037_P.csv a été créé et contient 7262 obseravtions.
Le fichier comb_00597X0007_S1.csv a été créé et contient 11323 obseravtions.
Le fichier comb_00603X0003_S1.csv a été créé et contient 11323 obseravtions.
Le fichier comb_00606X0086_P.csv a été créé et contient 5744 obseravtions.
Le fichier comb_00608X0206_S1.csv a été créé et contient 11323 obseravtions.
Le fichier comb

# Récupération manuelle

In [ ]:
reply = dict(request_id="54c52ee3-eee4-481d-b72e-fecd315085cb")
new_client = cdsapi.Client(wait_until_complete=False, delete=False)
result = cdsapi.api.Result(new_client, reply)
result.update()
reply = result.reply
if reply['state'] == 'completed':
    result.download('file.nc')

In [ ]:
requete_fichier_era5 = f"""
        UPDATE data_piezo
        SET fichier_era5 =
        (CASE WHEN code_bss IN {tuple(bss_era5)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_era5)

In [ ]:
update table set
    columnx = (case when condition then 25 else columnx end)